<a href="https://colab.research.google.com/github/Srinathshadow/Flipkart-Product-Reviews-Sentiment-Analysis-Using-Machine-Learning/blob/main/Flipkart_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import re
import nltk

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
df = pd.read_csv("/content/sample_data/Flipkart Reviews Sentiment Analysis.csv")

In [6]:
print(df.head())

   Unnamed: 0                                       Product_name  \
0           0  Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...   
1           1  Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...   
2           2  Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...   
3           3  DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...   
4           4  DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...   

                                              Review  Rating  
0  Best under 60k Great performanceI got it for a...       5  
1                                 Good perfomence...       5  
2  Great performance but usually it has also that...       5  
3           My wife is so happy and best product 👌🏻😘       5  
4  Light weight laptop with new amazing features,...       5  


In [7]:
df = df[['Review', 'Rating']]

In [8]:
df = df[df['Rating'] != 3]

In [9]:
df['Sentiment'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)

In [10]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['Cleaned_Review'] = df['Review'].apply(clean_text)

In [11]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Cleaned_Review']).toarray()
y = df['Sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [15]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9607390300230947

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.58      0.69        33
           1       0.97      0.99      0.98       400

    accuracy                           0.96       433
   macro avg       0.91      0.78      0.83       433
weighted avg       0.96      0.96      0.96       433



In [16]:
def predict_sentiment(review):
    review = clean_text(review)
    vector = vectorizer.transform([review]).toarray()
    prediction = model.predict(vector)
    return "Positive 😊" if prediction[0] == 1 else "Negative 😞"

In [17]:
print(predict_sentiment("The product quality is excellent and delivery was fast"))

Positive 😊
